In [8]:
import sys
sys.path.insert(0, '../util')
sys.path.insert(0, '../')
import matplotlib.pyplot as plt
from sc import *
from plot import *
from tqdm import tqdm
from alg import hstar, k_rollout, bhattacharya
from pqdict import pqdict
import numpy as np
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

# make scs

In [2]:
def generate_hole_sc(num_holes, hole_radius, border_eps, prox_eps, seed=1, max_iter = 1000):
    num_side = 17
    top, left, height, width = 1, -1, 2, 2

    # Initialize Simplicial Complex (SC)
    SC = SimplicialComplex(num_side, False, False, True)

    rng = np.random.Generator(np.random.PCG64(seed))
    centers = []
    bound = 1 - hole_radius - border_eps
    for k in range(num_holes):
        found_one, i = False, -1
        while (not found_one) and (i < max_iter):
            if i == max_iter:
                print(f"Max iter reached with {k} holes added. Change hole params!!")
                break
            i += 1
            x = np.round(rng.uniform(-bound, bound, 2), 1)

            if k == 0:
                centers.append(x)
                break
            
            too_close = False
            for y in centers:
                center_dist = np.linalg.norm(x-y, ord=np.inf)
                if (center_dist < hole_radius + prox_eps):
                    too_close = True
            if not too_close:
                centers.append(x)
                found_one = True

    centers = np.array(centers)
    centers -= np.mean(centers, axis=0)

    SC.make_holes(centers, hole_radius)

    return SC, centers

In [3]:
seed, hole_radius, border_eps, prox_eps = 12, 0.12, 0.12, 0.3
_, hole_locs_i = generate_hole_sc(9, hole_radius, border_eps, prox_eps, seed, 2000)
hole_locs = np.vstack([hole_locs_i, [-0.4, 0]])

sorted_idx = np.argsort(np.linalg.norm(hole_locs, 2, 1))
hole_locs = hole_locs[sorted_idx]

folder = "exp2_data"

In [ ]:
SCs = []
ref_outputs = []
coords = np.array([(-1,-1), (1, -1), (1, -0.7), (-1,0.7), (0,1), (1,1)])

for i in range(0, 10):
    num_side = 20
    SC_i = SimplicialComplex(num_side, False, False, True)
    all_faces = np.arange(SC_i.face_vec.shape[0])
    faces_to_add = {2:set(all_faces)}
    SC_i.add_simplices(faces_to_add)
    SC_i.make_holes(hole_locs[:i+1,:], hole_radius)
    ref_out = path_from_coords(SC_i, coords)
    SCs.append(SC_i)
    ref_outputs.append(ref_out)

In [ ]:
# for i, x in enumerate(SCs):
#     plot_path(x, ref_outputs[i], 'red', '')

In [ ]:
alphas = np.linspace(0, 3, 20)
np.savez(f"../{folder}/setup", alphas=alphas, SCs=SCs, ref_outputs=ref_outputs, hole_locs=hole_locs)

# Get data

In [9]:
folder = "../exp2_data"
results_dict = np.load(f"{folder}/setup.npz", allow_pickle=True)
alphas, SCs, ref_outputs, hole_locs = [results_dict[x] for x in results_dict.keys()]

In [10]:
eps = 1e-4
inputs1 = [(x[0], x[1], x[2], alphas, 0, False, eps, folder) for x in zip(range(len(SCs)), SCs, ref_outputs)]
inputs2 = [(x[0], x[1], x[2], alphas, 1, False, eps, folder) for x in zip(range(len(SCs)), SCs, ref_outputs)]
inputs3 = [(x[0], x[1], x[2], alphas, 1, True,  eps, folder) for x in zip(range(len(SCs)), SCs, ref_outputs)]

In [11]:
from wrapper import exp2_wrapper as wrap

In [ ]:
# # Base heuristic 
# pool = Pool(4)
# res = pool.map_async(wrap, inputs1).get()
# pool.terminate()

In [ ]:
# One step lookahead no pruning
pool = Pool(4)
res = pool.map_async(wrap, inputs2).get()
pool.terminate()

In [ ]:
# One step lookahead w/ pruning
pool = Pool(4)
res = pool.map_async(wrap, inputs2).get()
pool.terminate()

In [12]:
from wrapper import bhattacharya_wrapper as bhat_wrap

bhat_inputs = []
for i, (SC, ref_path, locs) in tqdm(enumerate(zip(SCs, ref_outputs, hole_locs))):
    start, end = ref_path[0], ref_path[-1]
    bhat_inputs.append([SC, ref_path, start, end, 1e-7, True, i, folder])

10it [00:00, 22882.18it/s]


In [13]:
pool = Pool(processes=4)
data_bhat = pool.starmap(bhat_wrap, bhat_inputs)
pool.terminate()

/Users/ctaveras/opt/anaconda3/envs/ot-simp/lib/python3.9/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)
/Users/ctaveras/opt/anaconda3/envs/ot-simp/lib/python3.9/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)
/Users/ctaveras/opt/anaconda3/envs/ot-simp/lib/python3.9/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarr